# Train MobileNet_V2 model with scooter image data


In [0]:
# Set location of training data and saved models on google drive

model_name = 'mobileV2_LA_0409'
data_path="LA_train2"  # training and validation data sets
#image_dir = 'drive/My Drive/CoLab_Data/Hollywood/'
image_dir = 'drive/My Drive/CoLab_Data/LosAngeles/'
saved_model_dir = 'drive/My Drive/CoLab_Data/save/'


In [0]:
#@title Load training data and configure workspace to train top layers of model

# define image size
IMAGE_SIZE = 224
BATCH_SIZE = 32
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# import dependencies
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
from numpy.random import rand
%tensorflow_version 2.x
import tensorflow as tf
from sklearn.utils import class_weight
from google.colab import files
from google.colab import drive

# connect to Google drive
drive.mount('/content/drive')

# unzip training images to local colab drive
os.system('unzip "%s%s.zip"' % (image_dir, data_path))

# Add random patch over 1/9 of image 50% of the time to training image generator
def addPatch(img):
  x = int(rand(1)[0]*3) * IMAGE_SIZE//3 
  y = int(rand(1)[0]*3) * IMAGE_SIZE//3 
  w = IMAGE_SIZE//3
  h = IMAGE_SIZE//3
  if rand(1)[0] > 0.5:
    for c in range(3):
      img[x:x+w,y:y+h,c]=255*rand(w,h)     
  return(img)

# Use ImageDataGenerator to rescale and augment the images.
data_path_train = data_path + '/train'
data_path_val = data_path + '/val'
datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    width_shift_range=[-0.15,0.15],
    height_shift_range=[-0.15,0.15],
    zoom_range=[0.9,1.1],
    rotation_range=10,
    preprocessing_function=addPatch) 

datagen_val = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = datagen_train.flow_from_directory(
    data_path_train,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True)

val_generator = datagen_val.flow_from_directory(
    data_path_val,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE)

# Setup callback to save intermediate results to Google drive every epoch
checkpoint_path = saved_model_dir + '/' + model_name + '_top/' +"cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')

# Define class labels
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

# Configure data generators
steps_per_epoch = train_generator.n // BATCH_SIZE
validation_steps = val_generator.n // BATCH_SIZE
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)

#class_weights = [1,class_weights[0], class_weights[1], class_weights[2]] # temp till get bike_lane and crosswalk data
#class_weights = [1, 1, class_weights[0], class_weights[1]] # temp till get bike_lane and crosswalk data
class_weights = dict(enumerate(class_weights))   
print(class_weights)

# Create the base model from the pre-trained model MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, layers=tf.keras.layers, 
                                              weights='imagenet')
base_model.trainable = False

# Add new top layer and compile
model = tf.keras.Sequential([
  base_model,
#  tf.keras.layers.Conv2D(32, 3, activation='relu'),
#  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(4, activation='softmax')  # first number specifies number of classes
])
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Found 23973 images belonging to 4 classes.
Found 9338 images belonging to 4 classes.
{0: 2.9079330422125182, 1: 3.8566602316602316, 2: 0.627959974853311, 3: 0.5542121324209358}
9412608/9406464 [==============================] - 0s 0us/step


# Either train top layers of model or load previous checkpoint

In [0]:
#Train top layers of model 

epochs = 5

# Initial training: Train just the top few layers of the model
history = model.fit(train_generator,
                    steps_per_epoch = steps_per_epoch, 
                    epochs=epochs,
                    callbacks=[cp_callback], 
                    validation_data=val_generator,
                    validation_steps=validation_steps,
                    class_weight=class_weights)


Epoch 1/5
749/749 [==============================] - ETA: 0s - loss: 0.6271 - accuracy: 0.7389
Epoch 00001: saving model to drive/My Drive/CoLab_Data/save//mobileV2_LA_0409_top/cp-0001.ckpt
749/749 [==============================] - 318s 425ms/step - loss: 0.6271 - accuracy: 0.7389 - val_loss: 0.4239 - val_accuracy: 0.8429
Epoch 2/5
749/749 [==============================] - ETA: 0s - loss: 0.4852 - accuracy: 0.7986
Epoch 00002: saving model to drive/My Drive/CoLab_Data/save//mobileV2_LA_0409_top/cp-0002.ckpt
749/749 [==============================] - 312s 416ms/step - loss: 0.4852 - accuracy: 0.7986 - val_loss: 0.4932 - val_accuracy: 0.8210
Epoch 3/5
749/749 [==============================] - ETA: 0s - loss: 0.4536 - accuracy: 0.8130
Epoch 00003: saving model to drive/My Drive/CoLab_Data/save//mobileV2_LA_0409_top/cp-0003.ckpt
749/749 [==============================] - 312s 417ms/step - loss: 0.4536 - accuracy: 0.8130 - val_loss: 0.3897 - val_accuracy: 0.8570
Epoch 4/5
749/749 [======

In [0]:
checkpoint = 'cp-0020.ckpt'
checkpoint_path = 'drive/My Drive/CoLab_Data/save/mobileV2_LA_0402_rev1'
model.load_weights(checkpoint_path + '/' + checkpoint)

In [0]:
# Configure model for fine tuning

# Unfreeze top layers of model
base_model.trainable = True
fine_tune_at = 100    
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

BATCH_SIZE = 128

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(lr=0.00002),
              metrics=['accuracy'])
model.summary()

# Setup data generators 
steps_per_epoch = train_generator.n // BATCH_SIZE
validation_steps = val_generator.n // BATCH_SIZE

datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    width_shift_range=[-0.15,0.15],
    height_shift_range=[-0.15,0.15],
    zoom_range=[0.9,1.1],
    rotation_range=25, 
    preprocessing_function=addPatch)

train_generator = datagen_train.flow_from_directory(
    data_path_train,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True)

# Setup callback to save intermediate results to Google drive every epoch
checkpoint_path = saved_model_dir + '/' + model_name + '/' +"cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 2,263,108
Trainable params: 1,867,716
Non-trainable params: 395,392
_________________________________________________________________
Found 23973 images belonging to 4 classes.


In [0]:
# Perform fine tuning

from sklearn.utils import class_weight

epochs=30

history_fine = model.fit(train_generator,
                         steps_per_epoch = steps_per_epoch,           
                         epochs=epochs,
                         callbacks=[cp_callback],
                         validation_data=val_generator,
                         validation_steps=validation_steps,
                         class_weight=class_weights)



In [0]:
 # Convert to TFlite, save and download results

 # Save model 
model.save(model_name + '.h5')
model.save(saved_model_dir + model_name + '.h5')
tf.saved_model.save(model, model_name)

# convert to tflite, original (pre2.0) method
#converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
#tflite_model = converter.convert()

####### Convert for 2.0 using concrete functions #####
# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : model(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Convert the model to tflite
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()
#open(TFLITE_MODEL, "wb").write(tflite_model)



# save tflite model to Google drive and download folder on local drive 
with open(saved_model_dir + model_name + '.tflite', 'wb') as f:
  f.write(tflite_model)
with open(model_name + '.tflite', 'wb') as f:
  f.write(tflite_model)
files.download(model_name + '.tflite')

In [0]:


files.download(model_name + '.tflite')